In [4]:
!pip install gtts
!pip install streamlit
!pip install pydub
#!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
#!pip install whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/

In [2]:
# Sample input
scenario_text = """Title is Operation Coastal Shield. Description is Allied Forces are retreating under fire.
Unit is ID equals U1. Name equals British Infantry. Type equals Infantry. Strength equals 85. Allegiance equals Friendly. X equals 3. Y equals 6. Status equals Active.
Unit is ID equals U2. Name equals French Infantry. Type equals Infantry. Strength equals 80. Allegiance equals Friendly. X equals 5. Y equals 6. Status equals Active.
Unit is ID equals U3. Name equals German Armor. Type equals Armor. Strength equals 92. Allegiance equals Enemy. X equals 3. Y equals 4. Status equals Active.
Unit is ID equals U4. Name equals German Armor. Type equals Armor. Strength equals 87. Allegiance equals Enemy. X equals 6. Y equals 4. Status equals Active.
Feature is Type equals Beach. X equals 5. Y equals 9.3. Size equals 50.
Objective is ID equals O1. Desk equals evacuate to Boats. X equals 5. Y equals 10. Priority equals 1.
Event is Time equals 0.00. Desk equals British Infantry fallback. Units equals U1. Type equals Move. X equals 5. Y equals 10.
Event is Time equals 0.01. Desk equals German Armor fires. Units equals U3. Type equals Fire. X equals 3. Y equals 10.
Event is Time equals 0.01. Desk equals French Infantry Fires. Units equals U2. Type equals Fire. X equals 5. Y equals 6."""

space_scenario_text = """Title is LUNAR ASCENT. Description is A covert laser strike has crippled UED’s high-bandwidth relay satellite over the Moon. Tensions escalate to open orbital warfare. The Moon becomes the fulcrum.
Unit is ID equals U1. Name equals UED Defense Satellite 1. Type equals Satellite. Strength equals 70. Allegiance equals Friendly. X equals 4. Y equals 2. Status equals Active.
Unit is ID equals U2. Name equals UED Defense Satellite 2. Type equals Satellite. Strength equals 70. Allegiance equals Home. X equals 3. Y equals 2.5. Status equals Active.
Unit is ID equals U3. Name equals UED Supply Shuttle. Type equals Shuttle. Strength equals 100. Allegiance equals Home. X equals 2. Y equals -1. Status equals Active.
Unit is ID equals U4. Name equals UED Artemis Laser. Type equals Ground-Laser. Strength equals 90. Allegiance equals Friendly. X equals 2. Y equals -2. Status equals Active.
Unit is ID equals U5. Name equals CC Stealth Satellite 1. Type equals StealthSatellite. Strength equals 60. Allegiance equals Enemy. X equals 3. Y equals 2.5. Status equals Active.
Unit is ID equals U6. Name equals CC Stealth Satellite 2. Type equals StealthSatellite. Strength equals 60. Allegiance equals Enemy. X equals 4. Y equals 2. Status equals Active.
Unit is ID equals U7. Name equals CC Lunar Jammer. Type equals Jammer. Strength equals 50. Allegiance equals Enemy. X equals 1. Y equals -5. Status equals Active.
Objective is ID equals O1. Desk equals Retake orbital sectors. X equals 0. Y equals 2. Priority equals 1.
Objective is ID equals O2. Desk equals Protect Artemis Base. X equals 0. Y equals 5. Priority equals 2.
Event is Time equals 1.00. Desk equals Deploy UED Defense Satellites. Units equals U1. Type equals Move. X equals 4. Y equals 2.
Event is Time equals 1.00. Desk equals Deploy UED Defense Satellites. Units equals U1. Type equals Move. X equals 7. Y equals 2.5.
Event is Time equals 2.00. Desk equals CC launches jammer activation. Units equals U7. Type equals Hold. X equals 4. Y equals 5.
Event is Time equals 3.00. Desk equals Direct fire between satellites begins. Units equals U1|U5. Type equals Fire. X equals 3. Y equals 2.5.
Event is Time equals 4.00. Desk equals UED Artemis Laser engages drones. Units equals U4|U8. Type equals Fire. X equals 2. Y equals 1.
Event is Time equals 5.00. Desk equals CC Stealth Satellite attempts repositioning. Units equals U6. Type equals Move. X equals 2. Y equals 3.
Event is Time equals 6.00. Desk equals UED Supply Shuttle prepares for descent. Units equals U3. Type equals Hold. X equals 0. Y equals 0.
Event is Time equals 7.00. Desk equals UED reclaims orbital sector. Units equals U2. Type equals Move. X equals 0. Y equals 2."""


narated_text = """In the shadow of the Moon, silence is shattered."

A precision laser strike has crippled the UED's high-bandwidth relay satellite, plunging orbital communications into chaos. The attack was unclaimed... but unmistakable.

Now, the conflict escalates.

Above the lunar surface, UED Defense Satellites scramble to reclaim strategic positions. Below, on the dark side of the Moon, Artemis Base powers up its ground-based laser array.

The Coalition of Colonies, long suspected of covert operations, activates stealth satellites and a powerful lunar jammer, blanketing the region in digital fog.

Objectives are clear:

Retake the orbital sectors.

Protect Artemis at all costs.

The battle for the Moon begins — and with it, the balance of power in space.

"This... is Lunar Ascent."

"""

In [5]:
%%writefile app.py
import streamlit as st
from gtts import gTTS
from pydub import AudioSegment
import tempfile
import os

st.title("🗣️ Generate Narration from Text")

text_input = st.text_area("🎤 Enter text to convert into speech:")

file_format = st.radio("Choose audio format:", options=["MP3", "WAV"])

if st.button("🔊 Generate Voice"):
    if not text_input.strip():
        st.warning("Please enter some text.")
    else:
        with st.spinner("Generating speech..."):
            tts = gTTS(text_input)
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as mp3_file:
                tts.save(mp3_file.name)
                mp3_path = mp3_file.name

            if file_format == "WAV":
                wav_path = mp3_path.replace(".mp3", ".wav")
                AudioSegment.from_mp3(mp3_path).export(wav_path, format="wav")
                audio_path = wav_path
            else:
                audio_path = mp3_path

        st.success("✅ Voice generated!")
        st.audio(audio_path)

        with open(audio_path, "rb") as f:
            st.download_button(f"⬇️ Download Narration ({file_format})", f.read(),
                               file_name=f"narration.{file_format.lower()}",
                               mime=f"audio/{file_format.lower()}")


Overwriting app.py


In [6]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 1s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.145.205.0
⠙your url is: https://free-ads-invent.loca.lt
